In [1]:
#importing useful packages
import pandas as pd
from google.colab import files
import io
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark-palette')
import datetime
import sklearn
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
import math
import re

In [2]:
#USE THIS CELL TO PULL DATA FROM GOOGLE DRIVE
#mount google colab
from google.colab import drive
drive.mount('/content/gdrive')

#import data from google drive
dummy = pd.read_csv('/content/gdrive/My Drive/urban economics/dummy_station_hrly.csv')
print(dummy.columns)
print(dummy.shape)
dummy.head()

Mounted at /content/gdrive
Index(['StationLength', 'Samples', '%Observed', 'TotalFlow', 'AvgOccupancy',
       'AvgSpeed', 'flowPerLane', 'DateTime', 'StationID_3009051',
       'StationID_3010011',
       ...
       'Freeway_65N', 'Freeway_65S', 'Freeway_70E', 'Freeway_70W',
       'Freeway_80E', 'Freeway_80W', 'Freeway_89N', 'Freeway_89S',
       'Freeway_99N', 'Freeway_99S'],
      dtype='object', length=686)
(442611, 686)


,StationLength,Samples,%Observed,TotalFlow,AvgOccupancy,AvgSpeed,flowPerLane,DateTime,StationID_3009051,StationID_3010011,StationID_3010012,StationID_3010091,StationID_3011031,StationID_3011041,StationID_3015021,StationID_3019101,StationID_3021122,StationID_3022031,StationID_3023021,StationID_3026051,StationID_3026052,StationID_3030131,StationID_3033061,StationID_3036042,StationID_3036072,StationID_3036081,StationID_3036091,StationID_3044012,StationID_3044021,StationID_3044091,StationID_3044092,StationID_3044101,StationID_3046032,StationID_3046042,StationID_3048041,StationID_3048042,StationID_3050011,StationID_3050031,StationID_3050041,StationID_3050081,...,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_3,Day_30,Day_31,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Freeway_12W,Freeway_160N,Freeway_160S,Freeway_267E,Freeway_267W,Freeway_28E,Freeway_28W,Freeway_50E,Freeway_50W,Freeway_51N,Freeway_51S,Freeway_5N,Freeway_5S,Freeway_65N,Freeway_65S,Freeway_70E,Freeway_70W,Freeway_80E,Freeway_80W,Freeway_89N,Freeway_89S,Freeway_99N,Freeway_99S
0,3.995,225,0,447.0,0.0245,63.6,223.500000,2020-10-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.883,356,100,905.0,0.0197,64.9,301.666667,2020-10-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.466,595,100,1105.0,0.0179,64.4,221.000000,2020-10-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.218,595,100,927.0,0.0141,67.4,185.400000,2020-10-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.483,476,100,542.0,0.0082,68.7,135.500000,2020-10-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
#we want to make a log of flow per lane variable, so we drop observations where flowPerLane = 0
dummy = dummy.loc[dummy['flowPerLane'] > 0]
dummy['lnFlowPerLane'] = np.log(dummy['flowPerLane'])
print(np.max(dummy['flowPerLane']))
print(np.max(dummy['lnFlowPerLane']))

4558.0
8.42463920980563


In [4]:
dummy_stationless = dummy.drop(dummy.filter(regex='^Stat').columns, axis=1)
dummy_stationless_freewayless = dummy_stationless.drop(dummy.filter(regex='^Freew').columns, axis=1)
dummy_stationless_freewayless.head()

,Samples,%Observed,TotalFlow,AvgOccupancy,AvgSpeed,flowPerLane,DateTime,numLanes_2.0,numLanes_3.0,numLanes_4.0,numLanes_5.0,numLanes_6.0,Hour_1,Hour_10,Hour_11,Hour_12,Hour_13,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_2,Hour_20,Hour_21,Hour_22,Hour_23,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,Hour_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_2,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_3,Day_30,Day_31,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,lnFlowPerLane
0,225,0,447.0,0.0245,63.6,223.500000,2020-10-01,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.409411
1,356,100,905.0,0.0197,64.9,301.666667,2020-10-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.709323
2,595,100,1105.0,0.0179,64.4,221.000000,2020-10-01,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.398163
3,595,100,927.0,0.0141,67.4,185.400000,2020-10-01,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.222516
4,476,100,542.0,0.0082,68.7,135.500000,2020-10-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.908972


In [5]:
#we rename the columns to make them easier to use
newcols = []
for var in dummy_stationless_freewayless.columns:
  newvar = re.sub(r'(Hour_)(.$)', r'\1 0\2', var)
  newvar = re.sub(r'(Day_)(.$)', r'\1 0\2', newvar)
  newvar = "".join(newvar.split())
  newcols.append(newvar)

print(newcols)
dummy_stationless_freewayless.columns = newcols


['Samples', '%Observed', 'TotalFlow', 'AvgOccupancy', 'AvgSpeed', 'flowPerLane', 'DateTime', 'numLanes_2.0', 'numLanes_3.0', 'numLanes_4.0', 'numLanes_5.0', 'numLanes_6.0', 'Hour_01', 'Hour_10', 'Hour_11', 'Hour_12', 'Hour_13', 'Hour_14', 'Hour_15', 'Hour_16', 'Hour_17', 'Hour_18', 'Hour_19', 'Hour_02', 'Hour_20', 'Hour_21', 'Hour_22', 'Hour_23', 'Hour_03', 'Hour_04', 'Hour_05', 'Hour_06', 'Hour_07', 'Hour_08', 'Hour_09', 'Day_10', 'Day_11', 'Day_12', 'Day_13', 'Day_14', 'Day_15', 'Day_16', 'Day_17', 'Day_18', 'Day_19', 'Day_02', 'Day_20', 'Day_21', 'Day_22', 'Day_23', 'Day_24', 'Day_25', 'Day_26', 'Day_27', 'Day_28', 'Day_29', 'Day_03', 'Day_30', 'Day_31', 'Day_04', 'Day_05', 'Day_06', 'Day_07', 'Day_08', 'Day_09', 'lnFlowPerLane']


In [6]:
X = dummy_stationless_freewayless.drop(columns=['Samples', '%Observed', 'TotalFlow', 'AvgOccupancy',
                                                'AvgSpeed', 'flowPerLane', 'DateTime', 'lnFlowPerLane'])
y = dummy_stationless_freewayless['lnFlowPerLane']


In [7]:
#we run a regression of X vars on ln flow per lane
reg_stless_frless = LinearRegression().fit(X,y)
print('Coefficient of determination R^2 of the prediction: {:.3f}'.format(reg_stless_frless.score(X,y)))

#rsf_coef = {}
#for coef, feat in zip(reg_stless_frless.coef_, X.columns):
#    rsf_coef[feat] = round(coef, 3)

rsf_coef = []
for i in range(len(reg_stless_frless.coef_)):
  rsf_coef.append([list(X.columns)[i], round(reg_stless_frless.coef_[i], 3)])

def custom_sort(t):
  return t[0]

rsf_coef.sort(key=custom_sort)

#rsf_coef.iterkeys()
print(rsf_coef)

Coefficient of determination R^2 of the prediction: 0.687
[['Day_02', 0.076], ['Day_03', -0.048], ['Day_04', -0.205], ['Day_05', -0.04], ['Day_06', -0.046], ['Day_07', -0.037], ['Day_08', -0.005], ['Day_09', 0.075], ['Day_10', -0.056], ['Day_11', -0.198], ['Day_12', -0.044], ['Day_13', -0.048], ['Day_14', -0.036], ['Day_15', -0.001], ['Day_16', 0.08], ['Day_17', -0.033], ['Day_18', -0.181], ['Day_19', -0.048], ['Day_20', -0.055], ['Day_21', -0.036], ['Day_22', -0.019], ['Day_23', 0.071], ['Day_24', -0.057], ['Day_25', -0.215], ['Day_26', -0.063], ['Day_27', -0.048], ['Day_28', -0.026], ['Day_29', -0.012], ['Day_30', 0.066], ['Day_31', -0.11], ['Hour_01', -0.283], ['Hour_02', -0.368], ['Hour_03', -0.219], ['Hour_04', 0.214], ['Hour_05', 0.881], ['Hour_06', 1.357], ['Hour_07', 1.577], ['Hour_08', 1.645], ['Hour_09', 1.685], ['Hour_10', 1.74], ['Hour_11', 1.804], ['Hour_12', 1.858], ['Hour_13', 1.883], ['Hour_14', 1.933], ['Hour_15', 1.984], ['Hour_16', 1.98], ['Hour_17', 1.934], ['Hour_1

In [10]:
from tabulate import tabulate
print(tabulate(rsf_coef, ["variable", "coefficient"], tablefmt="grid"))

+--------------+---------------+
| variable     |   coefficient |
+==============+===============+
| Day_02       |         0.076 |
+--------------+---------------+
| Day_03       |        -0.048 |
+--------------+---------------+
| Day_04       |        -0.205 |
+--------------+---------------+
| Day_05       |        -0.04  |
+--------------+---------------+
| Day_06       |        -0.046 |
+--------------+---------------+
| Day_07       |        -0.037 |
+--------------+---------------+
| Day_08       |        -0.005 |
+--------------+---------------+
| Day_09       |         0.075 |
+--------------+---------------+
| Day_10       |        -0.056 |
+--------------+---------------+
| Day_11       |        -0.198 |
+--------------+---------------+
| Day_12       |        -0.044 |
+--------------+---------------+
| Day_13       |        -0.048 |
+--------------+---------------+
| Day_14       |        -0.036 |
+--------------+---------------+
| Day_15       |        -0.001 |
+---------